In [21]:
from openai import OpenAI
import openai
import pandas as pd
client = OpenAI(api_key = 'OPENAI_API_KEY')

In [22]:
# Load the CSV file containing the input data
input_data = pd.read_csv("validated_CFA.csv")

In [41]:
los_data = pd.read_csv("Cleanedfinal_output.csv")

In [23]:
input_data.head()

,Name_of_the_topic,Year,Level,Introduction_Summary,Learning_Outcomes,Link_to_the_Summary_Page,Link_to_the_PDF_File,Summary
0,Time-Series Analysis,2024.0,2,"As financial analysts, we often use time-serie...",calculate and evaluate the predicted trend val...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN
1,Credit Analysis Models,2024.0,2,Credit analysis plays an important role in the...,"explain expected exposure, the loss given defa...",https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN
2,Introduction to Alternative Investments,2023.0,1,"In this section, we explain what alternative i...",describe types and categories of alternative i...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN
3,Credit Default Swaps,2024.0,2,Derivative instruments in which the underlying...,"describe credit default swaps (CDS), single-na...",https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN
4,Valuation of Contingent Claims,2024.0,2,A contingent claim is a derivative instrument ...,describe and interpret the binomial option val...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN


In [88]:
# Filter data for the specified topics
topics = [
    "Big Data Projects",
    "Backtesting & Simulation",
    "Branded Image Link Added to Refresher Reading"
]
filtered_data = input_data[input_data['Name_of_the_topic'].isin(topics)]
filtered_data.head()

,Name_of_the_topic,Year,Level,Introduction_Summary,Learning_Outcomes,Link_to_the_Summary_Page,Link_to_the_PDF_File,Summary
142,Branded Image Link Added to Refresher Reading,NaN,1,The authors of this article derive their formu...,Make sure you can add a BrandedImageLink compo...,https://www.cfainstitute.org/membership/profes...,NaN,"In today's competitive business environment, e..."


In [89]:
list_learning = {}
# for i in filtered_data:
#     print(i)
#     list_learning[i['Name_of_the_topic']] = i["Learning_Outcomes"].split(";")
# list_learning



In [43]:
# Filter data for the specified topics

In [73]:

# Function to generate technical notes for each LOS using GPT-3.5 Turbo
def generate_technical_notes(los, Summary):
    prompt = "Use the knowledge from {} to answer folllow up questions".format(Summary)
    query = "givme me detaild summery on the topic {} from you knowledge base return the response in markdown format".format(los)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        temperature=0,
    )
    return response.choices[0].message.content

In [90]:
import os

for index, row in filtered_data.iterrows():
    summary_new = row["Summary"]
    topic = row['Name_of_the_topic']
    learning_outcomes = row["Learning_Outcomes"].split(";")
    
    for los in learning_outcomes:
        res = generate_technical_notes(los, summary_new)
        filename = f"{topic}.md"
        filepath = os.path.join("~/Assignment5/notebooks/markdown/", filename)  # Specify your directory
        with open(filepath, 'w') as file:
            file.write(res)



In [91]:
import os

def read_markdown_files(directory_path):
    notes = {}
    for filename in os.listdir(directory_path):
        if filename.endswith(".md"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                notes[filename] = file.read()
    return notes


In [116]:

import pprint

def get_embeddings(text):
    response = client.embeddings.create(
        input=[text],
        model="text-embedding-ada-002"  # Or any model that suits your needs
    )
    
    pprint.pprint(vars(response))

    # embedding = response['data'][0]['embedding']
    # embedding = response['data'][0].embedding
    embedding = response.data[0].embedding  # Adjust based on actual structure


    return embedding


In [100]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [123]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="PINECONE_KEY")

index_name = "technical-notes"
if index_name not in pc.list_indexes():
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
 # Match the embedding size
index_pc = pc.Index(index_name)


In [124]:
directory_path = '~/Assignment5/notebooks/markdown/'

# Read markdown files
notes = read_markdown_files(directory_path)

# Process each note
for note_id, note_text in notes.items():
    vector = get_embeddings(note_text)
    index_pc.upsert(vectors=[(note_id, vector)])


{'data': [Embedding(embedding=[-0.008905963972210884, 0.030166013166308403, 0.009221874177455902, -0.023148786276578903, -0.02283959649503231, 0.008448903448879719, -0.004315191879868507, -0.00861021876335144, -0.011749151162803173, -0.04301748797297478, -0.005333496257662773, 0.03229000046849251, -0.04616314172744751, 0.01419577095657587, -0.006681153550744057, 0.003827884327620268, 0.027692507952451706, 0.010673713870346546, -0.006546724122017622, -0.03815113380551338, -0.024950142949819565, 0.006758450530469418, -0.003322092816233635, 0.002444939222186804, -0.009746149182319641, 0.0015694661997258663, 0.0186991635710001, -0.04325946047902107, -0.024923257529735565, 0.022705167531967163, 0.0029120822437107563, -0.001004861667752266, -0.018416861072182655, 0.0051284912042319775, -0.023471416905522346, 0.008146436884999275, 0.022960584610700607, -0.015607282519340515, 0.006123270373791456, -0.0025524829979985952, 0.006862633395940065, 0.007393630687147379, 0.0004894079174846411, -0.013